In [2]:
from statsmodels.graphics import utils
import missingno as msno
import fancyimpute

# %matplotlib inline

ModuleNotFoundError: No module named 'fancyimpute'

In [1]:
from statsmodels.graphics import utils
import missingno as msno
import fancyimpute
from fancyimpute import KNN
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
columns = {"parcelid": "I_parcelid" ,
"logerror": "Z_logerror" ,
"transactiondate": "T_transactiondate" ,
"airconditioningtypeid": "C_airconditioningtypeid" ,
"architecturalstyletypeid": "C_architecturalstyletypeid" ,
"basementsqft": "R_basementsqft" ,
"bathroomcnt": "R_bathroomcnt" ,
"bedroomcnt": "R_bedroomcnt" ,
"buildingclasstypeid": "C_buildingclasstypeid" ,
"buildingqualitytypeid": "C_buildingqualitytypeid" ,
"calculatedbathnbr": "R_calculatedbathnbr" ,
"decktypeid": "C_decktypeid" ,
"finishedfloor1squarefeet": "R_finishedfloor1squarefeet" ,
"calculatedfinishedsquarefeet": "R_calculatedfinishedsquarefeet" ,
"finishedsquarefeet12": "R_finishedsquarefeet12" ,
"finishedsquarefeet13": "R_finishedsquarefeet13" ,
"finishedsquarefeet15": "R_finishedsquarefeet15" ,
"finishedsquarefeet50": "R_finishedsquarefeet50" ,
"finishedsquarefeet6": "R_finishedsquarefeet6" ,
"fips": "C_fips" ,
"fireplacecnt": "R_fireplacecnt" ,
"fullbathcnt": "R_fullbathcnt" ,
"garagecarcnt": "R_garagecarcnt" ,
"garagetotalsqft": "R_garagetotalsqft" ,
"hashottuborspa": "C_hashottuborspa" ,
"heatingorsystemtypeid": "C_heatingorsystemtypeid" ,
"latitude": "R_latitude" ,
"longitude": "R_longitude" ,
"lotsizesquarefeet": "R_lotsizesquarefeet" ,
"poolcnt": "R_poolcnt" ,
"poolsizesum": "R_poolsizesum" ,
"pooltypeid10": "C_pooltypeid10" ,
"pooltypeid2": "C_pooltypeid2" ,
"pooltypeid7": "C_pooltypeid7" ,
"propertycountylandusecode": "C_propertycountylandusecode" ,
"propertylandusetypeid": "C_propertylandusetypeid" ,
"propertyzoningdesc": "C_propertyzoningdesc" ,
"rawcensustractandblock": "G_rawcensustractandblock" ,
"regionidcity": "C_regionidcity" ,
"regionidcounty": "C_regionidcounty" ,
"regionidneighborhood": "C_regionidneighborhood" ,
"regionidzip": "C_regionidzip" ,
"roomcnt": "R_roomcnt" ,
"storytypeid": "C_storytypeid" ,
"threequarterbathnbr": "R_threequarterbathnbr" ,
"typeconstructiontypeid": "C_typeconstructiontypeid" ,
"unitcnt": "R_unitcnt" ,
"yardbuildingsqft17": "R_yardbuildingsqft17" ,
"yardbuildingsqft26": "R_yardbuildingsqft26" ,
"yearbuilt": "T_yearbuilt" ,
"numberofstories": "R_numberofstories" ,
"fireplaceflag": "C_fireplaceflag" ,
"structuretaxvaluedollarcnt": "R_structuretaxvaluedollarcnt" ,
"taxvaluedollarcnt": "R_taxvaluedollarcnt" ,
"assessmentyear": "T_assessmentyear" ,
"landtaxvaluedollarcnt": "R_landtaxvaluedollarcnt" ,
"taxamount": "R_taxamount" ,
"taxdelinquencyflag": "C_taxdelinquencyflag" ,
"taxdelinquencyyear": "T_taxdelinquencyyear" ,
"censustractandblock": "G_censustractandblock" ,
}

In [4]:
# Initial data import
df_2016 = pd.read_csv("~/src/properties_2016.csv", index_col="parcelid")
train_2016 = pd.read_csv("~/src/train_2016.csv", index_col="parcelid")
train_2016_join = train_2016.join(df_2016)
train_2016_join.rename(columns=columns, inplace=True)
train_2016_join.sort_index(axis=1, inplace=True)
train_2016_join.to_csv("./datasets/train_2016_join.csv")

df_2017 = pd.read_csv("~/src/properties_2017.csv", index_col="parcelid")
train_2017 = pd.read_csv("~/src/train_2017.csv", index_col="parcelid")
train_2017_join = train_2017.join(df_2017)
train_2017_join.rename(columns=columns, inplace=True)
train_2017_join.sort_index(axis=1, inplace=True)
train_2017_join.to_csv("./datasets/train_2017_join.csv")

del df_2016
del df_2017

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# df_rs = pd.DataFrame()
# for x in train_2016_join.columns:
#     if x[0] == "R":
#         df_rs[x] = train_2016_join[x]
# names = dict(enumerate(list(df_rs.columns)))
# index = dict(enumerate(list(df_rs.index)))
# df_rs_impute = pd.DataFrame(fancyimpute.MICE(n_imputations=10).complete(df_rs))
# df_rs_impute.rename(columns=names, index=index, inplace=True)
# df_rs_impute["Z_logerror"] = train_2016_join["Z_logerror"]    

[MICE] Completing matrix with shape (90275, 30)
[MICE] Starting imputation round 1/20, elapsed time 0.057
[MICE] Starting imputation round 2/20, elapsed time 2.420
[MICE] Starting imputation round 3/20, elapsed time 4.650
[MICE] Starting imputation round 4/20, elapsed time 6.831
[MICE] Starting imputation round 5/20, elapsed time 9.039
[MICE] Starting imputation round 6/20, elapsed time 11.284
[MICE] Starting imputation round 7/20, elapsed time 13.571
[MICE] Starting imputation round 8/20, elapsed time 15.904
[MICE] Starting imputation round 9/20, elapsed time 18.308
[MICE] Starting imputation round 10/20, elapsed time 20.555
[MICE] Starting imputation round 11/20, elapsed time 22.758
[MICE] Starting imputation round 12/20, elapsed time 24.931
[MICE] Starting imputation round 13/20, elapsed time 27.148
[MICE] Starting imputation round 14/20, elapsed time 29.347
[MICE] Starting imputation round 15/20, elapsed time 31.553
[MICE] Starting imputation round 16/20, elapsed time 33.800
[MICE]

In [7]:
df_cs = pd.DataFrame()
for x in train_2016_join.columns:
    if x[0] != "R":
        df_cs[x] = train_2016_join[x]
df_cs["Z_logerror"] = df_rs_impute["Z_logerror"]

In [8]:
name1 = "C_propertycountylandusecode"
name2 = "C_regionidzip"
name3 = "C_taxdelinquencyflag"
name4 = "C_regionidneighborhood"
name5 = "T_taxdelinquencyyear"
name6 = "C_fips"
name7 = "C_heatingorsystemtypeid"
name8 = "C_buildingqualitytypeid"
name9 = "C_regionidcounty"
name10 = "C_regionidcity"
name11 = "C_propertyzoningdesc"

dic_1 = [
    "0100","0109","010C","010D","010E","0200","0401","040A","0700","1","1110",
    "1111","1129","112","1720","1722",]
dic_2 = [
    95982,95983,96005,96019,96123,96207,96267,96323,96354,96364,96464,
    96480,96488,96507,96951,96989,97003,97089,97118,97344
]
dic_4 = [
    31383,36630,40548,47880,115657,118920,268496,275340,275795,416314,416963,760999,
    762178,
]
dic_5 = [
    8.0, 14.0, 15.0
]
dic_6 = [
    6111.0
]
dic_7 = [
    2.0,
]
dic_8 = [
    1.0, 8.0
]
dic_10 = [
    3491.0, 6822.0, 10241.0, 12447.0, 12520.0, 13150.0, 13693.0, 18875.0, 20008.0,
    22827.0, 24245.0, 25621.0, 25953.0, 26965.0, 27110.0, 30267.0, 31134.0, 33612.0,
    33837.0, 34278.0, 36502.0, 37015.0, 38032.0, 39306.0, 40227.0, 45457.0, 46098.0,
    46298.0, 47568.0, 50749.0, 51239.0, 52650.0, 53027.0, 54053.0, 54212.0, 54299.0,
]
dic_11 = [
    u"ARR11000D*", u"AVU*", u"BFM1*",
    u"BHR1*", u"BRA2*", u"BUR5*", u"CCC3", u"CEADP4", u"CLHC*", 
    u"CUR3*", u"CV17500-RD", u"EMCO*", u"GLR-4*", u"HBR2B*", u"LAC2", u"LAC2(PV)", u"LAR3",
    u"LAR5", u"LARA", u"LARS", u"LBPD1", u"LBR2L", u"LCA11*", u"LCA11000*",
    u"LCA16000*", u"LCC1-RA100", u"LCC2YY", u"LCR110000*",
    u"LCR2YY", u"LCRA7S", u"LCRAL*", u"LCRPD13U-R", u"LPR2*",
    u"LRHI", u"LRP*", u"LRR 2 *", u"LUCR110000", u"MNRH", u"MNRM", 
    u"PDM4", u"POC4-R3100", u"POR3*", u"PSR1", u"RMC3D*", u"RMR1YY", u"SGR3-R1*", 
    u"SGR305", u"SLR1YY", u"SOR12L", u"SRRPCC*", u"SSR1*", u"TCR4YY", u"TORR-LO", u"WAR18500RU", u"WDC1B*"
]

df_cs[name1].where(df_cs[name1].isin(dic_1), other="Others", inplace=True)
df_cs[name2].where(df_cs[name2].isin(dic_2), other="Others", inplace=True)
df_cs[name4].where(df_cs[name4].isin(dic_4), other="Others", inplace=True)
df_cs[name5].where(df_cs[name5].isin(dic_5), other="Others", inplace=True)
df_cs[name6].where(df_cs[name6].isin(dic_6), other="Others", inplace=True)
df_cs[name7].where(df_cs[name7].isin(dic_7), other="Others", inplace=True)
df_cs[name8].where(df_cs[name8].isin(dic_8), other="Others", inplace=True)
df_cs[name10].where(df_cs[name10].isin(dic_10), other="Others", inplace=True)
df_cs[name11].where(df_cs[name11].isin(dic_11), other="Others", inplace=True)

In [ ]:
df_join = df_rs_impute.join(df_cs.drop("Z_logerror", axis=1))

name1 = "C_propertycountylandusecode"
name2 = "C_regionidzip"
name3 = "C_taxdelinquencyflag"
name4 = "C_regionidneighborhood"
name5 = "T_taxdelinquencyyear"
name6 = "C_fips"
name7 = "C_heatingorsystemtypeid"
name8 = "C_buildingqualitytypeid"
name9 = "C_regionidcounty"
name10 = "C_regionidcity"
name11 = "C_propertyzoningdesc"

c_features = [
    "R_logerror", name1, name2, name3, name4, name5, name6, name7, name8, name9,
    name10, name11
]

reals = ["scale({})".format(x) for x in df_join.columns if x[0] == "R"]
cates = ["C({})".format(x) for x in c_features[1:]]
features = []
for x in cates:
    features.append(x)
for x in reals:
    features.append(x)
    for y in cates:
        features.append("{}:{}".format(x, y))

formula = "Z_logerror~"+"+".join(features)

df_OLS = df_join.fillna("Others")
model = sm.OLS.from_formula(formula, df_OLS)
result = model.fit()
result.summary()